In [2]:
import nltk
import time
from collections import Counter
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV

import csv
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import regexp_tokenize, wordpunct_tokenize,blankline_tokenize
from nltk import PorterStemmer, LancasterStemmer, SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
import re
import string
from collections import Counter
import json
import re as regex
import xgboost as xgb
from sklearn import model_selection, preprocessing
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
from sklearn.metrics import r2_score


In [3]:
class TwitterData_Initialize():
    data = []
    processed_data = []
    wordlist = []

    featureList = []
    fea_vect=[]
    
    data_model = None
    data_labels = None
    is_testing = False
    
    def initialize(self, csv_file, is_testing_set=False, from_cached=None):
        if from_cached is not None:
            #self.data_model = pd.read_csv(from_cached)
            self.data_model = pd.read_json(from_cached)

            return

        self.is_testing = is_testing_set

        if not is_testing_set:
            #self.data = pd.read_csv(csv_file, header=0, names=["id", "emotion", "text"])
            self.data = pd.read_json(csv_file)

            #self.data = self.data[self.data["emotion"].isin(["positive", "negative", "neutral"])]
        

        self.processed_data = self.data
        self.wordlist = []
        self.data_model = None
        self.data_labels = None
        
    
    
    
    def do_process(self):
        start_time = time.time()
        def stem_and_join(row,stemmer=nltk.PorterStemmer()):
            row["spans"] = list(map(lambda str: stemmer.stem(str.lower()), row["spans"]))
            return row
    
        def tokenize_grams(row):
                
                # turn a doc into clean tokens
                def clean_doc(doc):
                    # split into tokens by white space
                    tokens = doc.split()
                    # remove punctuation from each token
                    table = str.maketrans('', '', string.punctuation)
                    tokens = [w.translate(table) for w in tokens]
                    # remove remaining tokens that are not alphabetic
                    tokens = [word for word in tokens if word.isalpha()]
                    # filter out stop words
                    #whitelist=["to","on","n't","not","don't","for","up","below","short","long"]
                    whitelist=[]
                    stop_words = set(stopwords.words('english'))
                    
                    tokens = [w for w in tokens if (not w in stop_words or w in whitelist)]
                    # filter out short tokens
                    tokens = [word.lower() for word in tokens if len(word) > 1]
                    return tokens
                




               
                # Function to apply lemmatization to a list of words
                def words_lemmatizer(words, encoding="utf8"):
                    wordnet_lemmatizer = WordNetLemmatizer()

                    lemma_words = []
                    wl = WordNetLemmatizer()
                    for word in words:
                        pos = find_pos(word)
                        lemma_words.append(wl.lemmatize(word, pos))
                    return lemma_words
                
                # Function to find part of speech tag for a word
                def find_pos(word):
                    # Part of Speech constants
                    # ADJ, ADJ_SAT, ADV, NOUN, VERB = 'a', 's', 'r', 'n', 'v'
   
                    pos = nltk.pos_tag(nltk.word_tokenize(word))[0][1]
                    # Adjective tags - 'JJ', 'JJR', 'JJS'
                    if pos.lower()[0] == 'j':
                        return 'a'
                    # Adverb tags - 'RB', 'RBR', 'RBS'
                    elif pos.lower()[0] == 'r':
                        return 'r'
                    # Verb tags - 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'
                    elif pos.lower()[0] == 'v':
                        return 'v'
                    # Noun tags - 'NN', 'NNS', 'NNP', 'NNPS'
                    else:
                        return 'n'
    
            
                #convert to string
                idx=row["spans"]
                #ch="".join(x for x in idx if x)
                ch=' '.join(idx)
               
    
                
                
                n_grams=clean_doc(ch)
                tokens=words_lemmatizer(n_grams)

                #print("nombre of tokens",len(n_grams))

                row["token_spans"] = tokens
    
                return row
        #self.processed_data = self.processed_data.apply(stem_and_join, axis=1)
        self.processed_data = self.processed_data.apply(tokenize_grams, axis=1)
        print("--- %s seconds ---" % (time.time() - start_time))

    

    def build_wordlist(self, min_occurrences=0, max_occurences=500, stopwords=nltk.corpus.stopwords.words("english"),
                       ):
       
        self.wordlist = []
        whitelist=[]
        
        stopwords=[]
       
        words = Counter()
      
        for idx in self.processed_data.index:
            words.update(self.processed_data.loc[idx, "token_spans"])

        for idx, stop_word in enumerate(stopwords):
            if stop_word not in whitelist:
                del words[stop_word]
        
        
        print(words.most_common())        

        word_df = pd.DataFrame(data={"word": [k for k, v in words.most_common() if min_occurrences < v < max_occurences],
                                     "occurrences": [v for k, v in words.most_common() if min_occurrences < v < max_occurences]},
                               columns=["word", "occurrences"])

        word_df.to_csv("wordlist_copie.csv", index_label="idx")
        self.wordlist = [k for k, v in words.most_common() if min_occurrences < v < max_occurences]
        
        
        
    def build_data_model(self):
        label_column = []
        Id_column=["ID"]
        label_column = ["label"]

        columns = Id_column + label_column + list(
            map(lambda w: w ,self.wordlist))
        labels = []
        rows = []
        for idx in self.processed_data.index:
            current_row = []

            if True:
                # add label
                current_label = self.processed_data.loc[idx, "sentiment score"]
                current_id = self.processed_data.loc[idx, "id"]
                
                labels.append(current_id)
                labels.append(current_label)
                
                current_row.append(current_id)
                current_row.append(current_label)

            # add bag-of-words
            tokens = set(self.processed_data.loc[idx, "token_spans"])
            for _, word in enumerate(self.wordlist):
                current_row.append(1 if word in tokens else 0)

            rows.append(current_row)

        self.data_model = pd.DataFrame(rows, columns=columns)
        self.data_labels = pd.Series(labels)
        return self.data_model, self.data_labels


In [4]:
class TwitterData_Initialize_test(TwitterData_Initialize):

    
    
    def do_process(self):
        def stem_and_join(row,stemmer=nltk.PorterStemmer()):
            
            row["spans"] = list(map(lambda str: stemmer.stem(str.lower()), row["spans"]))
            return row
    
        def tokenize_grams(row):
                
                # Function to remove stop words
                def remove_stopwords(text, lang='english'):
                    whitelist = ["n't","not","below"]    

                    stop_words = set(stopwords.words('english'))
                    word_tokens = word_tokenize(text)
                    #filtered_sentence = [w for w in word_tokens if ((not w in stop_words) or (w in whitelist))]
                    filtered_sentence = []
                    for w in word_tokens:
                        if ((w not in stop_words) or (w in whitelist)):
                            filtered_sentence.append(w)
                
                    ch=" ".join(filtered_sentence)

                    return ch
#40

                def clean_doc(doc):
                    # split into tokens by white space
                    tokens = doc.split()
                    # remove punctuation from each token
                    table = str.maketrans('', '', string.punctuation)
                    tokens = [w.translate(table) for w in tokens]
                    # remove remaining tokens that are not alphabetic
                    tokens = [word for word in tokens if word.isalpha()]
                    # filter out stop words
                    #whitelist=["to","on","n't","not","don't","for","up","below","short","long"]
                    whitelist=[]
                    stop_words = set(stopwords.words('english'))
                    
                    tokens = [w for w in tokens if (not w in stop_words or w in whitelist)]
                    # filter out short tokens
                    tokens = [word.lower() for word in tokens if len(word) > 1]
                    return tokens


                

                # Function to apply lemmatization to a list of words
                def words_lemmatizer(words, encoding="utf8"):
                    wordnet_lemmatizer = WordNetLemmatizer()

                    lemma_words = []
                    wl = WordNetLemmatizer()
                    for word in words:
                        pos = find_pos(word)
                        lemma_words.append(wl.lemmatize(word, pos))
                    return lemma_words

                # Function to find part of speech tag for a word
                def find_pos(word):
                    # Part of Speech constants
                    # ADJ, ADJ_SAT, ADV, NOUN, VERB = 'a', 's', 'r', 'n', 'v'
   
                    pos = nltk.pos_tag(nltk.word_tokenize(word))[0][1]
                    # Adjective tags - 'JJ', 'JJR', 'JJS'
                    if pos.lower()[0] == 'j':
                        return 'a'
                    # Adverb tags - 'RB', 'RBR', 'RBS'
                    elif pos.lower()[0] == 'r':
                        return 'r'
                    # Verb tags - 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'
                    elif pos.lower()[0] == 'v':
                        return 'v'
                    # Noun tags - 'NN', 'NNS', 'NNP', 'NNPS'
                    else:
                        return 'n'
    
            
                token=[]
                idx=row["spans"]
                ch=' '.join(idx)


    
                #words_stemmer(ch, type="PorterStemmer", lang="english", encoding="utf8")
                #Convert to lower case
                n_grams=clean_doc(ch)
                tokens=words_lemmatizer(n_grams)

                #print("list ngrams--------------\n",n_grams)
                row["token_spans"] = tokens
    
                return row
        #self.processed_data = self.processed_data.apply(stem_and_join, axis=1)
        self.processed_data = self.processed_data.apply(tokenize_grams, axis=1)
        
    def build_wordlist(self, min_occurrences=0, max_occurences=500, stopwords=nltk.corpus.stopwords.words("english"),
                      ):
        
        whitelist=[]
        self.wordlist = []
        stopwords=[]
        words = Counter()
      
        for idx in self.processed_data.index:
            words.update(self.processed_data.loc[idx, "token_spans"])

        for idx, stop_word in enumerate(stopwords):
            if stop_word not in whitelist:
                del words[stop_word]
        
        
        print(words.most_common())        

        word_df = pd.DataFrame(data={"word": [k for k, v in words.most_common() if min_occurrences < v < max_occurences],
                                     "occurrences": [v for k, v in words.most_common() if min_occurrences < v < max_occurences]},
                               columns=["word", "occurrences"])

        word_df.to_csv("wordlist_test.csv", index_label="idx")
        self.wordlist = [k for k, v in words.most_common() if min_occurrences < v < max_occurences]
        
        
    def build_data_model(self):
        
        label_id = ["ID"]

        columns = label_id + list(
            map(lambda w: w ,self.wordlist))
        labels = []
        rows = []
        for idx in self.processed_data.index:
            current_row = []
            if True:
                # add label
                current_id = self.processed_data.loc[idx, "id"]
                
                labels.append(current_id)
                
                current_row.append(current_id)

            

            # add bag-of-words
            tokens = set(self.processed_data.loc[idx, "token_spans"])
            for _, word in enumerate(self.wordlist):
                current_row.append(1 if word in tokens else 0)

            rows.append(current_row)

        self.data_model = pd.DataFrame(rows, columns=columns)
        return self.data_model


In [5]:
data = TwitterData_Initialize()
data.initialize("Microblog_Trainingdata.json")

data.processed_data.head(20)

,cashtag,id,sentiment score,source,spans
0,$FB,719659409228451840,0.366,twitter,[watching for bounce tomorrow]
1,$LUV,719904304207962112,0.638,twitter,[record number of passengers served in 2015]
2,$NFLX,5329774,-0.494,stocktwits,[out $NFLX -.35]
3,$DIA,719891468173844480,0.460,twitter,"[Looking for a strong bounce, Lunchtime rally ..."
4,$PLUG,20091246,0.403,stocktwits,[Very intrigued with the technology and growth...
5,$GMCR,5819749,0.000,stocktwits,"[short worked, puts up]"
6,$IBM,709741154393133056,-0.296,twitter,[overbought]
7,$JOSB,17892972,-0.546,stocktwits,"[absolute garbage still up, stores TOTALLY EMP..."
8,$CSTM,709834259687710720,-0.438,twitter,[Biggest Market Losers]
9,$PYPL,708481442079068160,0.408,twitter,[Love this company long time.]


In [6]:
data.do_process()

--- 195.05737805366516 seconds ---


In [7]:
data.processed_data.head()

,cashtag,id,sentiment score,source,spans,token_spans
0,$FB,719659409228451840,0.366,twitter,[watching for bounce tomorrow],"[watch, bounce, tomorrow]"
1,$LUV,719904304207962112,0.638,twitter,[record number of passengers served in 2015],"[record, number, passenger, serve]"
2,$NFLX,5329774,-0.494,stocktwits,[out $NFLX -.35],[nflx]
3,$DIA,719891468173844480,0.460,twitter,"[Looking for a strong bounce, Lunchtime rally ...","[look, strong, bounce, lunchtime, rally, come]"
4,$PLUG,20091246,0.403,stocktwits,[Very intrigued with the technology and growth...,"[very, intrigue, technology, growth, potential]"


In [8]:
data_test = TwitterData_Initialize_test()
data_test.initialize("Microblog_Trialdata2.json")

data_test.processed_data.head()
#test_data=data_test.processed_data.copy()
data_test.do_process()
data_test.processed_data.head()


,cashtag,id,sentiment score,source,spans,token_spans
0,$F,5540055,-0.454,stocktwits,[Putting on a little $F short],"[put, little, short]"
1,$AAPL,10752226,-0.464,stocktwits,[short some],[short]
2,$BAC,10920221,0.445,stocktwits,[buying opportunity],"[buying, opportunity]"
3,$SHOR,12971398,0.661,stocktwits,[Scaling Up on Long Position],"[scale, up, long, position]"
4,$JPM,16142438,-0.763,stocktwits,[its time to sell banks],"[time, sell, bank]"


In [9]:
"""# Importing the built-in logging module
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
"""

"# Importing the built-in logging module\nimport logging\nlogging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)\n"

In [20]:
#list_tweet=get_tweets(data)
list_tweet=list(data.processed_data["token_spans"])
#sentences=review_sentences(list_tweet, remove_stopwords=True)
sentences=list_tweet
taille=0
for tokens in sentences: 
    taille+=len(tokens)
print(taille)


7096


In [11]:
"""# Creating the model and setting values for the various parameters
num_features = 200  # Word vector dimensionality
min_word_count = 2 # Minimum word count
num_workers = 4     # Number of parallel threads
context = 10        # Context window size
downsampling = 1e-3 # (0.001) Downsample setting for frequent words

# Initializing the train model
from gensim.models import word2vec
print("Training model....")
w2v_model = word2vec.Word2Vec(sentences,\
                          workers=num_workers,\
                          size=num_features,\
                          min_count=min_word_count,\
                          window=context,
                          sample=downsampling)

# To make the model memory efficient
w2v_model.init_sims(replace=True)

# Saving the model for later use. Can be loaded using Word2Vec.load()
model_name = "microblogs_word_vector"
w2v_model.save(model_name)
#model = gensim.models.Word2Vec.load(model_name)
"""

'# Creating the model and setting values for the various parameters\nnum_features = 200  # Word vector dimensionality\nmin_word_count = 2 # Minimum word count\nnum_workers = 4     # Number of parallel threads\ncontext = 10        # Context window size\ndownsampling = 1e-3 # (0.001) Downsample setting for frequent words\n\n# Initializing the train model\nfrom gensim.models import word2vec\nprint("Training model....")\nw2v_model = word2vec.Word2Vec(sentences,                          workers=num_workers,                          size=num_features,                          min_count=min_word_count,                          window=context,\n                          sample=downsampling)\n\n# To make the model memory efficient\nw2v_model.init_sims(replace=True)\n\n# Saving the model for later use. Can be loaded using Word2Vec.load()\nmodel_name = "microblogs_word_vector"\nw2v_model.save(model_name)\n#model = gensim.models.Word2Vec.load(model_name)\n'

In [12]:
"""# Few tests: This will print the odd word among them 
w2v_model.wv.doesnt_match("Thanks for the short upper".split())
len(w2v_model.wv.vocab)"""

'# Few tests: This will print the odd word among them \nw2v_model.wv.doesnt_match("Thanks for the short upper".split())\nlen(w2v_model.wv.vocab)'

In [13]:
"""w2v_model.wv.most_similar("big")
#len(model.wv.index2word)
list(map(lambda i: "word2vec_{0}".format(i), range(0, w2v_model.vector_size)))"""

'w2v_model.wv.most_similar("big")\n#len(model.wv.index2word)\nlist(map(lambda i: "word2vec_{0}".format(i), range(0, w2v_model.vector_size)))'

In [14]:
import gensim
from gensim.models import KeyedVectors
# Load Google's pre-trained Word2Vec model.
goo_model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)  
#goo_model=gensim.models.Word2Vec.load('GoogleNews-vectors-negative300.bin')

C:\Users\Globalnet\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [15]:

import numpy as np
# Function to average all word vectors in a paragraph
def featureVecMethod(words, model, num_features):
    # Pre-initialising empty numpy array for speed
    featureVec = np.zeros(num_features,dtype="float32")
    nwords = 0
    
    #Converting Index2Word which is a list to a set for better speed in the execution.
    index2word_set = set(model.wv.index2word)
    for word in  words:
        if word in index2word_set:
            nwords = nwords + 1
            featureVec = np.add(featureVec,model[word])
    
    # Dividing the result by number of words to get average
    featureVec = np.divide(featureVec, nwords)
    return featureVec

In [16]:
# Function for calculating the average feature vector
def getAvgFeatureVecs(reviews, model, num_features):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    for review in reviews:
        # Printing a status message every 1000th review
        if counter%1000 == 0:
            print("Review %d of %d"%(counter,len(reviews)))
            
        reviewFeatureVecs[counter] = featureVecMethod(review, model, num_features)
        counter = counter+1
        
    return reviewFeatureVecs

In [27]:
#goo_model.wv.most_similar("the")
print(goo_model.wv['the'])
len(goo_model.wv.vocab)
num_features=300


[ 0.08007812  0.10498047  0.04980469  0.0534668  -0.06738281 -0.12060547
  0.03515625 -0.11865234  0.04394531  0.03015137 -0.05688477 -0.07617188
  0.01287842  0.04980469 -0.08496094 -0.06347656  0.00628662 -0.04321289
  0.02026367  0.01330566 -0.01953125  0.09277344 -0.171875   -0.00131989
  0.06542969  0.05834961 -0.08251953  0.0859375  -0.00318909  0.05859375
 -0.03491211 -0.0123291  -0.0480957  -0.00302124  0.05639648  0.01495361
 -0.07226562 -0.05224609  0.09667969  0.04296875 -0.03540039 -0.07324219
  0.03271484 -0.06176758  0.00787354  0.0035553  -0.00878906  0.0390625
  0.03833008  0.04443359  0.06982422  0.01263428 -0.00445557 -0.03320312
 -0.04272461  0.09765625 -0.02160645 -0.0378418   0.01190186 -0.01391602
 -0.11328125  0.09326172 -0.03930664 -0.11621094  0.02331543 -0.01599121
  0.02636719  0.10742188 -0.00466919  0.09619141  0.0279541  -0.05395508
  0.08544922 -0.03686523 -0.02026367 -0.08544922  0.125       0.14453125
  0.0267334   0.15039062  0.05273438 -0.18652344  0.

C:\Users\Globalnet\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
C:\Users\Globalnet\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [28]:
#transformer les donner de train en moyenne de vecteurs word2vec 
trainDataVecs = getAvgFeatureVecs(list_tweet, goo_model, num_features)

Review 0 of 1700


C:\Users\Globalnet\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Globalnet\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide


Review 1000 of 1700


In [30]:
X_train=trainDataVecs


array([[-0.03369141,  0.05423991,  0.09147135, ..., -0.03519694,
         0.0546875 , -0.16031902],
       [ 0.11431885, -0.05249023,  0.12463379, ...,  0.04917145,
         0.04614258, -0.19024658],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       ...,
       [ 0.036026  , -0.04504395, -0.00216293, ...,  0.0113678 ,
         0.06007195, -0.05272675],
       [ 0.03466797,  0.1184082 , -0.14648438, ..., -0.1875    ,
         0.01748657, -0.20214844],
       [-0.5859375 , -0.27148438, -0.62109375, ...,  0.38867188,
         0.25      ,  0.07080078]], dtype=float32)

In [32]:
y_train=data.processed_data["sentiment score"].values


In [33]:
#X_train, X_test, y_train, y_test = train_test_split(X, Y,train_size=0.7)                                                  


In [34]:
#---------------------------XGBregressor---------------------------------------------------

#Fitting XGB regressor with parameters obtained by Grid searchCV
"""
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
param_test2 = {
 'max_depth':[4,5,6],
 'min_child_weight':[4,5,6]
}
param_test3 = {
    'gamma':[i/10.0 for i in range(0,5)],
    'n_estimators' : [int(x) for x in np.linspace(start = 200, stop = 1400, num = 7)]
}

n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
model = GridSearchCV(estimator = XGBRegressor( learning_rate =0.1, max_depth=4,
 min_child_weight=4,  subsample=0.8, colsample_bytree=1.0, nthread=4), 
 param_grid = param_test3,scoring='neg_mean_squared_error',cv=5)
model.fit(X_train,y_train)

"""
#model = xgb.XGBRegressor(max_depth=4, min_child_weight= 4,gamma=0.3,subsample=0.8,colsample_bytree=1.0,n_estimators=1000)
XGBR=xgb.XGBRegressor(learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 nthread=4,
 scale_pos_weight=1,
 seed=27)
XGBR.fit(X_train,y_train)

print (XGBR)


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=4, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=True,
       subsample=0.8)


In [ ]:
#------------------RandomForest--------------------------
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
"""
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train,y_train)

"""
# Import the model we are using
# Instantiate model with 1000 decision trees
RF = RandomForestRegressor(random_state = 42,bootstrap= True,
 max_depth= None,
 max_features= 'sqrt',
 min_samples_leaf= 1,
 min_samples_split= 10,
 n_estimators= 1600)
# Train the model on training data
RF.fit(X_train, y_train);
print (RF)


In [ ]:
#---------------------------------DecisionTree----------------------------
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
# set of parameters to test
param_grid = {
              "min_samples_split": [2, 10, 20],
              "max_depth": [None, 2, 5, 10],
              "min_samples_leaf": [1, 5, 10],
              "max_leaf_nodes": [None, 5, 10, 20],
              }

model = DecisionTreeRegressor()
DTR = GridSearchCV(model, param_grid)
DTR.fit(X_train, y_train)
#accuracy = [cross_val_score(model, X_train, y_train, cv=8, n_jobs=-1)]

In [35]:
#test model accuracy
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error


print("mean squared error:" ,mean_squared_error(XGBR.predict(X_train), y_train))
print("R2 score:" ,r2_score(XGBR.predict(X_train),y_train))

mean squared error: 0.005772898478391591
R2 score: 0.9569408884704746


In [36]:
data_test.do_process()
data_test.processed_data.head()

,cashtag,id,sentiment score,source,spans,token_spans
0,$F,5540055,-0.454,stocktwits,[Putting on a little $F short],"[put, little, short]"
1,$AAPL,10752226,-0.464,stocktwits,[short some],[short]
2,$BAC,10920221,0.445,stocktwits,[buying opportunity],"[buying, opportunity]"
3,$SHOR,12971398,0.661,stocktwits,[Scaling Up on Long Position],"[scale, up, long, position]"
4,$JPM,16142438,-0.763,stocktwits,[its time to sell banks],"[time, sell, bank]"


In [37]:
test_tweet=list(data_test.processed_data["token_spans"])

In [38]:
testDataVecs = getAvgFeatureVecs(test_tweet, goo_model, num_features)

Review 0 of 16


C:\Users\Globalnet\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # Remove the CWD from sys.path while we load stuff.


In [39]:
X_test=testDataVecs
X_test[0].size

300

In [40]:
#isoler les tweets du test et leur vrai valeur de sentiment 
labels=["real pred","spans"]
pred=list(data_test.processed_data["sentiment score"])
twt=list(data_test.processed_data["spans"])



In [41]:
#Predict 
output_2 = XGBR.predict(X_test)
test_df = pd.DataFrame(pred,columns={"real prediction"})
#final_df["ID"] = id_vals
#final_df["cashtag"]=row2

test_df["spans"]=twt
test_df["Prediction"] = output_2
#final_df.to_csv("Output_1.csv",sep=",")
test_df.head(20)

,real prediction,spans,Prediction
0,-0.454,[Putting on a little $F short],-0.259628
1,-0.464,[short some],-0.502769
2,0.445,[buying opportunity],0.348896
3,0.661,[Scaling Up on Long Position],0.487097
4,-0.763,[its time to sell banks],-0.748225
5,0.627,[Entering long],0.276600
6,0.653,[picked some up],0.210565
7,0.668,"[time to accumulate for a long position, far m...",0.624018
8,0.460,"[Looking for a strong bounce, Lunchtime rally ...",0.473562
9,0.403,[Very intrigued with the technology and growth...,0.399923


In [42]:
#tester la performance de prediction avec R2 score et MSE
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

print("mean squared error:" ,mean_squared_error(output_2, pred))
print("R2 score:" ,r2_score(output_2,pred))

mean squared error: 0.033261851149993055
R2 score: 0.8115568040385457
